# Decomposition of a general single--qutrit gate
> And a couple of words about Euler's and KAK decompositions
- toc: true 
- badges: true
- comments: true
- categories: [quantum concepts, compilation]
- image: images/relative_toffoli.png

In [3]:
#collapse
import numpy as np
from scipy.stats import unitary_group

# What this post is about

The main purpose of this post is to give an explicit formula for decomposition of an arbitrary single-qutrit gate. If you want to cut to the chase, proceed to [summary and implementation](#summary-and-implementation). The idea is explaine asdfThe decomposition is drawn in [here](#decomposition-refinenmet) and sloppily implemented [here](#implementation).

For context I also cover Euler's decomposition of the single-qubit gates and give a brief motivation below.

So, why care about qutrits? Well, first thing to note is that they exist. In the real world, in contrast to most quantum textbooks, physical system are not just two-level but can have many, or even infinite amount of levels. One prominent example is cold atoms

# Euler's decomposition of an arbitrary single-qubit gate

## Elementary single-qubit rotations
Me (and probably you too) usually take for granted that an arbitrary single-qubit gate $U$ can be decomposed into a product of three elementary rotations. The rotations are usually chosen as

\begin{align}
&R_X(\theta)=e^{-i\theta X/2}=\begin{pmatrix} \cos\frac{\theta}2 & -i\sin\frac{\theta}2 \\  -i\sin\frac{\theta}2 & \cos\frac{\theta}2\end{pmatrix}\\
&R_Y(\theta)=e^{-i\theta Y/2}=\begin{pmatrix} \cos\frac{\theta}2 & -\sin\frac{\theta}2 \\  \sin\frac{\theta}2 & \cos\frac{\theta}2\end{pmatrix}\\
&R_Z(\theta)=e^{-i\theta Z/2}=\begin{pmatrix} e^{-i \theta/2} & 0 \\ 0 & e^{i\theta/2}\end{pmatrix}
\end{align}

and a sample decomposition reads

$$U = R_X(\theta_1)R_Z(\theta_2)R_X(\theta_3) \label{XZX}$$

This is known as the Euler decomposition. Choosing angles $\theta_i$ appropriately one can obtain any $2\times 2$ special (with unit determinant) unitary matrix $U\in SU(2)$. 

## Making sense of Euler's decomposition
Let me make a couple of remarks in an attempt to make relation \eqref{XZX} a bit more intuitive. First of all, the dimension of the group $U(n)$, i.e. a group of  $n\times n$ unitary matrices is equal to
$$\operatorname{dim} U(n)=n^2 \ .$$
If you also factor out the global phase, i.e. impose $\operatorname{det}U=1$ you get the special unitary group $SU(n)$ which has one dimension less
$$\operatorname{dim} SU(n)=n^2-1\ .$$
For $n=2$, corresponding to a single-qubit gate, $\operatorname{dim}=3$. This implies that the right-hand side in eq.\eqref{XZX} needs to have at least three parameters and hence three elementary rotations. Good, this explains why are there three terms in Euler's decomposition. (This technique of unsubtle parameter counting in fact extends far more generally. For example, it can be used to deduce a minimum amount of $CNOT$ gates necessary to compile an arbitrary $n$-qubit unitary, as I discussed [here](https://idnm.github.io/blog/machine%20learning/compilation/qiskit/paper%20review/2021/07/22/Machine-learning-compilation-of-quantum-circuits.html#Theoretical-lower-bound-and-quantum-Shannon-decomposition)).

## A word about Cartan decomposition (KAK)
Now, is there anything special about this particular combination of rotations $XZX$? Well, yes and no. In fact, almost any three single-parameter unitaries would work. Other possible decompositions are $XYX$, $YZY$ etc, but also less common options exist such as $XYZ$. 

However, decompositions like $XZX$ are special in a sense that they are examples of Cartan (aka KAK) decompositions, hinging on the structure of the underlying $su(2)$ algebra. Decompositions of two-qubit gates into $CNOT$ gates and single-qubit rotations is another important example where KAK arise. I will not cover the subject here, but can recommend a quite readable introduction with applications to quantum gates [https://arxiv.org/abs/quant-ph/0010100](https://arxiv.org/abs/quant-ph/0010100).


## Finding angles in Euler's decomposition

I have not told you yet how to find angles in the Euler decomposition, so here we go. The key observation is that because $XZ=-XZ$ we have
$$X R_Z(\theta)X=R_Z(-\theta),\quad X R_X(\theta)X=R_X(\theta) \ .$$
Multiplying \eqref{XZX} by $X$ from left and right we get
$$XUX=R_X(\theta_1)R_Z(-\theta_2)R_X(\theta_3) \ .$$
Therefore,
$$M:= UXU^\dagger X=R_x(\theta_1)R_Z(2\theta_2)R_x^\dagger(\theta_1) \ .$$

Since $R_Z$ is a diagonal gate, the right hand side is nothing else but a unitary diagonalization of matrix $M$. Therefore, finding $\theta_1$ and $\theta_2$ reduces to the standard diagonalization procedure. The last parameter $\theta_3$ is then read off from $R_X(\theta_3)=R_Z^\dagger(\theta_2) R_X^\dagger(\theta_1)U$. 


Here is a quick and dirty code that implements the procedure. It won't handle some edge cases and may not be the most efficient, but I'll be happy with it's performance on random unitaries.

In [4]:
# Define X, RX, and RZ.

X = np.array([[0, 1], [1, 0]])

def RX(theta):
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)],[-1j*np.sin(theta/2), np.cos(theta/2)]])

def RZ(theta):
    return np.array([[np.exp(-1j*theta/2), 0],[0, np.exp(1j*theta/2)]])

# Recovering angle from a cos and sine.
def angle_from_cos_and_sin(c, s):
    assert np.allclose(c**2+s**2, 1), 'Input does not satisfy c**2+s**2=1'
    phi = np.arccos(c)
    if np.allclose(np.sin(phi), s):
        return phi
    else:
        return -phi
        

# Decomposition routine.

def XZX_decomposition(U):
    
    assert np.allclose(U.conj().T @ U, np.eye(2)), 'Input matrix is not unitary.'
    assert np.allclose(np.linalg.det(U), 1), 'Input matrix has non-unit determinant.'
    
    
    # Construct the matrix to diagonalize.
    M = U @ X @ U.conj().T @ X
    
    # Diagonalization.
    RZ_squared, RX1 = np.linalg.eig(M)
    assert np.allclose(RX1 @ np.diag(RZ_squared) @ RX1.conj().T, M)
    
    # Matrix RX1 is not necessarily of RX(theta) form yet, but it can be made so by multiplying with a diagonal unitary.
    phase_00 = RX1[0, 0]/np.abs(RX1[0, 0])
    RX1 /= phase_00 # Now RX1[0,0] is real and RX[1,0] has the correct phase.
    phase_10 = RX1[1, 0]/np.abs(RX1[1, 0])
    phase_01 = RX1[0, 1]/np.abs(RX1[0, 1])
    
    RX1 = RX1 @ np.diag([1, phase_10/phase_01]) # Now RX1[0,1] and RX1[1,1] have the correct phase as well.
    
    # Determine theta_1.
    cos = RX1[0, 0]
    sin = 1j*RX1[0, 1]
    theta_1 = 2*angle_from_cos_and_sin(cos, sin)
    assert np.allclose(RX(theta_1), RX1)

    # Theta_2 from RZ squared.
    theta_2 = 1j*np.log(RZ_squared[0]) 
    assert np.allclose(RZ(theta_2), RZ(theta_2))
        
    # Determine RX(theta_3) and theta_3. 
    RX2 = RZ(-theta_2) @ RX(-theta_1) @ U
    cos = RX2[0, 0]
    sin = 1j*RX2[0, 1]
    theta_3 = 2*angle_from_cos_and_sin(cos, sin)
    
    assert np.allclose(RX(theta_3), RX2)

    if np.allclose(RX(theta_1) @ RZ(theta_2) @ RX(theta_3), U):
        return theta_1, theta_2, theta_3

    raise TypeError('Something went wrong during decomposition.')

And now let's test.

In [5]:
# Draw a random unitary and normalize its det to 1.
U = unitary_group.rvs(2, random_state=0) 
U = U / np.linalg.det(U)**(1/2) 

# Decompose and check
theta_1, theta_2, theta_3 = XZX_decomposition(U)
check = np.allclose(RX(theta_1) @ RZ(theta_2) @ RX(theta_3), U)

print(f'Does it work? {check}')

Does it work? True


From $XZX$ decompositions we can get related ones. For example, since $HXH=Z$ and $HZH=X$ we $XZX$ decomposition for $HUH$ is in fact the $ZXZ$ decomposition for $U$. We'll need it a bit later so let's implement that one also.

In [6]:
def ZXZ_decomposition(U):
    H = np.array([[1, 1], [1, -1]])/np.sqrt(2)
    return XZX_decomposition(H@U@H)

A simple check.

In [7]:
U = unitary_group.rvs(2, random_state=2)
U /= np.sqrt(np.linalg.det(U))

theta_1, theta_2, theta_3 = ZXZ_decomposition(U)
print('Check:', np.allclose(RZ(theta_1) @ RX(theta_2) @ RZ(theta_3), U))

Check: True


# Decomposing arbitrary single-qutrit gate

## Elementary qutrit gates
Qutrit is an abstraction for any three-level quantum system and it's is probably less familiar than a qubit. General qutrit gate belongs to $SU(3)$ and elementary rotation gates for a qutrit are commonly chosen as 
\begin{align}
R^{01}_X(\theta)=\begin{pmatrix}\cos\frac{\theta}2&-i\sin\frac{\theta}2&0\\-i\sin\frac{\theta}2&\cos\frac{\theta}2&0\\0&0&1\end{pmatrix},\quad 
R^{01}_Z(\theta)=\begin{pmatrix}e^{-i\theta/2}&0&0\\0&e^{i\theta/2}&0\\0&0&1\end{pmatrix} \ ,
\end{align}

and

\begin{align}
R^{12}_X(\theta)=\begin{pmatrix}1&0&0\\0&cos\frac{\theta}2&-i\sin\frac{\theta}2\\0&-i\sin\frac{\theta}2&\cos\frac{\theta}2\end{pmatrix},\quad 
R^{12}_Z(\theta)=\begin{pmatrix}1&0&0\\0&e^{-i\theta/2}&0\\0&0&e^{i\theta/2}\end{pmatrix} \ .
\end{align}

Similar expression hold to $R_Y^{01}$ and $R_Y^{12}$ but we won't need them. And by the way gates that act only on levels 1 and 3 look bit awkward, and happily we can avoid them.
## Is there a an off-the-shelf result? (I'm bad at googling)
A general single-qutrit transformation lives in $SU(3)$ and hence has $3^2-1=8$ real parameters. Thus, we expect to have a decomposition which is a product of 8 elementary rotations. But how exactly will it look, and how to determine the parameters? To my surprise, the only explicit result I found is equation (4) in this paper [https://arxiv.org/abs/1105.5485](https://arxiv.org/abs/1105.5485) which goes something like
$$U = R_Y^{01}R_Y^{02}R_Y^{01}R_Z^{01}R_Z^{02}R_Y^{01}R_Y^{02}R_Y^{01} \ .$$

The result is based on a particular Cartan decomposition of $SU(3)$ (which is non-unique) and does not look very intuitive. Moreover, apparently no explicit algorithm to find the parameters of these rotations (which I omitted in the equation) is given.

## Decomposition sketch
I will now give another explicit decomposition for a general single-qutrit gate with elementary derivation and explicit procedure to compute the parameters. I begin with separating general $U\in SU(3)$ into four blocks
$$U=\begin{pmatrix}A&B\\C&D\end{pmatrix}=\begin{pmatrix} a_{00}& a_{01} & b_0\\ a_{10} & a_{11} & b_1\\ c_0&c_1&d\end{pmatrix} \ .$$

Here $A$ is a $2\times2$ matrix, $B$ and $C^\dagger$ are column vectors and $D$ is a scalar. Now let us factorize matrix $A$ using the [singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD) 
$$A = V \Sigma W \ . \label{A SVD}$$
Here $V$ and $W$ are unitary, while $\Sigma=\operatorname{diag}(\sigma_0,\sigma_1)$ is diagonal with $\sigma_i\ge0$. SVD is a very useful  factorization that works for arbitrary matrices, not necessarily hermitian or unitary (or even square). Using SVD of $A$ we can factorize $U$ as follows
$$U=\begin{pmatrix}V&0\\0&1\end{pmatrix}\begin{pmatrix}\Sigma&V^\dagger B\\ CW^\dagger&D\end{pmatrix}\begin{pmatrix}W&0\\0&1\end{pmatrix}=V^{01}\begin{pmatrix}\Sigma&V^\dagger B\\ CW^\dagger&D\end{pmatrix}W^{01} \label{U part} \ .$$
The left and right matrices act only within the first two levels. The middle matrix looks like it acts on all levels of the qutrit, but in fact it doesn't. I will leave at as an exercise to show that matrix $A$ necessarily has a unit singular value (Hint: unitarity condition for $U$ implies $A^\dagger A+C^\dagger C=\mathbb{1}$. Because $C$ is a vector $C^\dagger C$ is not full rank and has a zero eigenvalue). I will assume that $\sigma_{0}=1$. This has consequences. In a unitary matrix each row and column has a unit norm. Therefore, if some entry in a unitary matrix is equal to one, all other elements in the corresponding row and column must be zero. Thus, the middle matrix in \eqref{U part} in fact looks like
$$H^{12}:=\begin{pmatrix}1&0&0\\ 0&\sigma_{1}& b'\\ 0&c'&d\end{pmatrix}$$
and hence only acts on levels 1 and 2. The overall decomposition can be sketched as follows.

<img src="myimages/qutrit/qutrit.png" alt="Drawing" style="width: 400px;"/>

Needless to say that the lines here aren't qubits, they are individual levels of the qutrit. 

Therefore, we have split a general qutrit gate into three gates touching two levels only. Each of these can be further decomposed into three elementary rotations using Euler's method for single-qubit gates. That's good. However, as it stands the decomposition is not optimal.  So far all matrices $V, H, W$ could lie in $U(2)$ and hence have four real parameters. We should be able to do better.

## A word about global phases
Sure, global phases are normally ignored, and rightfully so, but there are situations where they are important. In our case, a global phase for a two-qubit gate leads to a non-trivial transformation when embedded in a qutrit. For example,
$$(e^{i\phi})^{01}=\begin{pmatrix}e^{i\phi}&0&0\\0&e^{i\phi}&0\\0&0&1\end{pmatrix}$$
is surely not a global phase for a qutrit. You may try to decompose this gate into the elementary qutrit rotations introduced above+a true global phase gate for a qutrit. The result won't be trivial and actually involve more than one qutrit rotation. Therefore, we will take extra care to avoid global phases in the two-level gates.

## Decomposition refinement
To refine the decomposition we will use redundancies to impose additional constraints on matrices $V,H,W$. First, let us write $V$ and $W$ in the $ZXZ$ representation $V=e^{i\phi_V}Z_{1V}X_{V}Z_{2V}, W=e^{i\phi_W}Z_{1W}X_{W}Z_{2W}$ and substitute this into the SVD of $A$ \eqref{A SVD}. After grouping together diagonal terms in the middle we get
$$A=V'\left(Z_{1V}X_V\tilde{Z}\right)\left(\tilde{Z}^\dagger e^{i\phi_V}e^{i\phi_W}Z_{2V}Z_{1W}\Sigma\right)\left(X_{W}Z_{2W}\right)$$
Here $\tilde{Z}$ is an additional $R_Z$ gate that so far is arbitrary. We will choose it so that middle matrix is of the form $\Sigma'=\operatorname{diag}(1,\sigma_1e^{i\phi})$ for some phase $\phi$. We therefore obtain a new factorization
$$A=V'\Sigma'W'$$
where both $V'$ and $W'$ have unit determinant and, moreover, $W'$ has only two degrees of freedom $W'=XZ$ (of course one could remove this degree of freedom from $V'$ instead of $W'$). Note that $\Sigma'$ now has a complex eigenvalue, but this will not be a problem, the important thing is that $\Sigma'_{00}=1$. This new factorization of $A$ leads to the following decomposition of $U$
$$U=(V')^{01} (H')^{12}(W')^{12}$$
Because $\operatorname{det}U=1$ it follows $\operatorname{det}H'=1$. Therefore, the middle term also needs only three gates to be decomposed.

## Summary and implementation
In the end, we get the following parameter-optimal decomposition of the form
<img src="myimages/qutrit/decomposition.png" alt="Drawing" style="width: 600px;"/>

The algorithm can be summarized as follows.
1. Pick $2\times2$ submatrix $A$ of unitary $U$ and find its singular value decomposition $A=V \Sigma W$.
1. Factorize $V$ and $W$ via Euler's ZXZ decomposition $V=e^{i\phi_V}Z_{V1} X_{V} Z_{V2}$, $W=e^{i\phi_W}Z_{W1} X_{W} Z_{W2}$.
1. Define 
\begin{align}
&V'=Z_{V1} X_V (Z_{V2}\tilde{Z})\\
&\Sigma'=e^{i\phi_V}e^{i\phi_W}\widetilde{Z}^\dagger \Sigma Z_{W1}\\
&W'=X_W Z_{W2}
\end{align}
Choose $\widetilde{Z}$ so that $\Sigma'_{00}=1$.
1. Compute $V'^\dagger U W'^\dagger$. It will be of the form $\begin{pmatrix}1&0&0\\0&H'_{00}&H'_{01}\\0&H'_{10}&H'_{11} \end{pmatrix}$ for some matrix $H'\in SU(2)$.
1. Decompose $H'$ via your favorite single-qubit factorization, say $ZXZ$ and voilà.


Let's go ahead and implement this decomposition. Again, I will not care about efficiency or potential edge cases.

In [8]:
# Embedding qubit gates into qutrit.

def embed(U_qubit, levels):
    U = np.eye(3, dtype=np.complex64)
    if levels == '01':
        U[:2, :2]=U_qubit
    elif levels == '12':
        U[1:, 1:]=U_qubit
    else:
        raise TypeError(f'Levels {levels} not supported.')
        
    return U

# Decomposition routine

def qutrit_decomposition(U):
    assert np.allclose(U.conj().T @ U, np.eye(3)), 'Input in not unitary.' 
    assert np.allclose(np.linalg.det(U), 1), 'Input is not special.'
    
    
    A = U[:2, :2] # 2x2 submatrix
    V, Sigma, W = np.linalg.svd(A) # Basic SVD
    
    # Setting det V = det W = 1
    phase_V = np.sqrt(np.linalg.det(V))
    phase_W = np.sqrt(np.linalg.det(W))
    
    V /= phase_V
    W /= phase_W
    
    # RZ factor to remove from W
    theta_W1, theta_W2, theta_W3 = ZXZ_decomposition(W)
    ZW1 = RZ(theta_W1)
    
    # Computing the middle diagonal matrix
    M = phase_V*phase_W*np.diag(Sigma) @ ZW1
    M00_phase = -1j*np.log(M[0, 0])
    
    # Gauge factor to set M[0,0]=1
    Z_gauge = RZ(2*M00_phase)
    
    # Final matrices
    V_prime = V @ Z_gauge.conj().T
    W_prime = RX(theta_W2) @ RZ(theta_W3)
    H_prime_full = embed(V_prime.conj().T, '01') @ U @ embed(W_prime.conj().T, '01')
    H_prime = H_prime_full[1:, 1:]
    
    theta_V1, theta_V2, theta_V3 = ZXZ_decomposition(V_prime)
    theta_H1, theta_H2, theta_H3 = ZXZ_decomposition(H_prime)
    
    return theta_V1, theta_V2, theta_V3, theta_H1, theta_H2, theta_H3, theta_W2, theta_W3

And, let's verify.

In [9]:
U = unitary_group.rvs(3, random_state=0)
U = U/np.linalg.det(U)**(1/3)

theta = qutrit_decomposition(U)

V01 = embed(RZ(theta[0]) @ RX(theta[1]) @ RZ(theta[2]), '01')
H12 = embed(RZ(theta[3]) @ RX(theta[4]) @ RZ(theta[5]), '12')
W01 = embed(RX(theta[6]) @ RZ(theta[7]), '01')

print('Does it work?', np.allclose(V01 @ H12 @ W01, U))

Does it work? True
